## Imports

In [41]:
import os
import re
import sys
import typing
import gc

sys.path.append(
    os.path.join('..','src')
)

from src.models import NextWordPredictorModel
from src.data_processing import *

In [42]:
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

import torch
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [43]:
import nltk

from nltk.corpus import webtext
# from nltk.tokenize import TweetTokenizer
from nltk.tokenize import sent_tokenize

## Global Variables

In [44]:
DEVICE = "cuda:0"
assert torch.cuda.is_available()
from apex import amp, optimizers

PADDING_TOKEN = 'PAD' # voc 0
UNKNOWN_TOKEN = 'UKN' # voc 1

MAX_SEQ_LEN = 20

BATCH_SIZE = 16
EMB_SIZE = 256

val_split = 0.2
test_split = 0.1

# for reproducibility
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
torch.manual_seed(0)
np.random.seed(23)

## Vocabulary

In [45]:
train_iter = WikiText2(split='train')
tokenizer = get_tokenizer('basic_english')

train_text = ' '.join(train_iter)

In [46]:
vocabulary = FromRawTextVocabulary(
    text = train_text,
    tokenizer = tokenizer,
    text_cleaner = None,
    max_voc_size = 10000,
    min_word_occ = 10
)

## Dataset

In [47]:
train_iter, val_iter, test_iter = WikiText2()

train_text = ' '.join(train_iter)
val_text = ' '.join(val_iter)
test_text = ' '.join(test_iter)

train_dataset = SequenceDataset(
    vocabulary = vocabulary,
    text = train_text,
    max_seq_length = MAX_SEQ_LEN + 1,
    device = DEVICE
)
val_dataset = SequenceDataset(
    vocabulary = vocabulary,
    text = val_text,
    max_seq_length = MAX_SEQ_LEN + 1,
    device = DEVICE
)
test_dataset = SequenceDataset(
    vocabulary = vocabulary,
    text = test_text,
    max_seq_length = MAX_SEQ_LEN + 1,
    device = DEVICE
)

del train_text
del val_text
del test_text

gc.collect()

0

In [48]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = True,
    pin_memory = False,
    drop_last = True
)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    pin_memory = False,
    drop_last = True
)

In [51]:
def map_weights(weights, m_ = 0.1, M_ = 1):
    weights = 1 / weights
    M, m = max(weights), min(weights)
    return (np.array(weights) - m) * (M_ - m_) / (M - m) + m_

weights = map_weights(np.array(list(vocabulary.vocab.values())))
#weights = None

In [53]:
model = NextWordPredictorModel(
    emb_dim  = EMB_SIZE,
    vocab_size = vocabulary.get_vocab_size(),
    num_lstm_hidden_layers = 2,
    hidden_state_size = 200,
    dropout = 0.5,
    device = DEVICE,
    lr = 1e-3,
    fp16 = True,
    weight = weights
).to(DEVICE)

if model.fp16:
    model, model.optimizer = amp.initialize(
        model,
        model.optimizer,
        opt_level = 'O1' # https://nvidia.github.io/apex/amp.html
    )

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


In [54]:
metrics = model.fit(
    train_dataloader = train_dataloader,
    eval_dataloader = val_dataloader,
    num_epochs = 100,
    early_stopping = True,
    early_stopping_patience = 2,
    early_stopping_metric = 'val_loss',
    early_stopping_metric_best = 'min', # if lower is better (like for loss)
)

  0%|          | 13/7076 [00:00<00:57, 122.27it/s]

Train loss at epoch 0 : 9.208509548024447
Eval loss at epoch 0 : 9.20819157731501
updating best metric


  0%|          | 12/7076 [00:00<01:00, 116.78it/s]

Train loss at epoch 1 : 8.769418183324564
Eval loss at epoch 1 : 7.661188159027511
updating best metric


 13%|█▎        | 947/7076 [00:07<00:47, 128.35it/s]

...

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 524288.0


  0%|          | 14/7076 [00:00<00:53, 131.25it/s]

Train loss at epoch 34 : 7.129407206317541
Eval loss at epoch 34 : 6.830802625080325
updating best metric


 11%|█         | 792/7076 [00:05<00:46, 134.41it/s]


KeyboardInterrupt: 

In [55]:
metrics

{0: {'train_loss': 9.211254693074979,
  'val_loss': 9.211471538338056,
  'lr': 0.01},
 1: {'train_loss': 7.481150862791903,
  'val_loss': 6.791930608672273,
  'lr': 0.0099},
 2: {'train_loss': 7.032236319062668,
  'val_loss': 6.694209414351018,
  'lr': 0.009801},
 3: {'train_loss': 6.965919334447741,
  'val_loss': 6.655035271477506,
  'lr': 0.00970299},
 4: {'train_loss': 6.935509015571741,
  'val_loss': 6.638524394151014,
  'lr': 0.0096059601},
 5: {'train_loss': 6.917404641735817,
  'val_loss': 6.628796935402801,
  'lr': 0.009509900499},
 6: {'train_loss': 6.905118976965539,
  'val_loss': 6.624028734119754,
  'lr': 0.00941480149401},
 7: {'train_loss': 6.896221652820731,
  'val_loss': 6.622018424005843,
  'lr': 0.0093206534790699},
 8: {'train_loss': 6.8900200257940165,
  'val_loss': 6.614226840577036,
  'lr': 0.0092274469442792},
 9: {'train_loss': 6.884402382838787,
  'val_loss': 6.61305994036384,
  'lr': 0.009135172474836408},
 10: {'train_loss': 6.880649321290853,
  'val_loss': 6

In [56]:
sent = 'so how are'
ind = [vocabulary.word_to_idx[w] for w in sent.split(' ')]
print(ind)
hidden = model.init_hidden(1)
inputs = torch.tensor([ind]).to(DEVICE)
model.eval()
output, hidden = model(inputs, hidden)
preds = output.view(-1, model.vocab_size)

[126, 372, 27]


In [57]:
for i in preds.topk(1)[1].view(-1):
    w = vocabulary.idx_to_word[i.item()]
    print(w)

the
the
the


In [58]:
preds.topk(10)

torch.return_types.topk(
values=tensor([[2.8789, 2.4219, 2.3633, 2.3262, 2.2344, 2.2227, 2.1797, 1.9902, 1.8594,
         1.8535],
        [4.5586, 3.9941, 3.9199, 3.8750, 3.7578, 3.7402, 3.6875, 3.4121, 3.2031,
         3.1836],
        [5.7578, 5.1094, 5.0234, 4.9805, 4.8281, 4.8203, 4.7461, 4.4062, 4.1367,
         4.0977]], device='cuda:0', dtype=torch.float16,
       grad_fn=<TopkBackward>),
indices=tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 12],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 12],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 12]], device='cuda:0'))